In [3]:
import numpy as np
from astropy.io import fits
from astropy.io.fits import HDUList
from astropy.wcs import WCS
from numpy import arcsinh
import matplotlib.pyplot as plt
from astropy.visualization import astropy_mpl_style
from reproject import reproject_interp
import aplpy
from aplpy.rgb import make_rgb_cube
import matplotlib.pyplot as pyplot
from PIL import Image
plt.rcParams.update(plt.rcParamsDefault)

In [5]:
%ls -l data/

total 1611656
-rw-r--r--@ 1 will  staff  349816837 11 Mar 15:14 astromonical_data.csv.gz
drwxr-xr-x  4 will  staff        128  6 Apr 16:18 fits/
-rw-r--r--  1 will  staff   12447360  6 Apr 16:52 frame-g-000756-2-0427.fits
-rw-r--r--  1 will  staff   12447360  6 Apr 16:52 frame-i-000756-2-0427.fits
-rw-r--r--  1 will  staff   12447360  6 Apr 16:52 frame-r-000756-2-0427.fits
-rw-r--r--@ 1 will  staff  319952110 31 Jan 20:01 input.csv
-rw-r--r--  1 will  staff   36979200 11 Feb 19:42 rgb-000756-2-0427.fits
-rw-r--r--  1 will  staff   12329280 11 Feb 19:42 rgb-000756-2-0427_2d.fits
-rw-r--r--  1 will  staff   36933120 20 Jan 17:16 rgb-002141-3-0076.fits
-rw-r--r--  1 will  staff   12314880 20 Jan 17:16 rgb-002141-3-0076_2d.fits


In [6]:
fits_files = ['./data/frame-i-000756-2-0427.fits', './data/frame-r-000756-2-0427.fits', './data/frame-g-000756-2-0427.fits']

In [7]:
output = './data/rgb-000756-2-0427.fits'

In [8]:
make_rgb_cube(fits_files, output)

ValueError: More than one HDU is present, please specify HDU to use with ``hdu_in=`` option

In [ ]:
# Make an RGB image
aplpy.make_rgb_image('rgb-000756-2-0427.fits', 'rgb-000756-2-0427.png', stretch_r='arcsinh', stretch_g='arcsinh', stretch_b='arcsinh')

# Plot the RGB image using the 2d image to indicate the projection
f = aplpy.FITSFigure('rgb-000756-2-0427_2d.fits')
f.show_rgb('rgb-000756-2-0427.png')

In [ ]:
twod_fits = fits.open('rgb-000756-2-0427_2d.fits')

In [ ]:
twod_fits[0].header

In [ ]:
%ls -l

In [ ]:
a = fits.open('rgb-000756-2-0427.fits')

In [ ]:
a_hdu = a[0]
a_header = a_hdu.header
a_data = a_hdu.data

In [ ]:
a_header

In [ ]:
a_header['NAXIS'] = 2

In [ ]:
#a_data = np.nan_to_num(a_data + a_data.min()) - a_data.min()
# a_data[0] = convolve(a_data[0], kernel)
# a_data[1] = convolve(a_data[1], kernel)
# a_data[2] = convolve(a_data[2], kernel)

In [ ]:
n, x_shape, y_shape = a_data.shape

In [ ]:
def modify_image(data, gains=[0.9,1.1,1.8], gamma=0.01):
    R_IDX = 0
    G_IDX = 1
    B_IDX = 2

    r = data[:,:,R_IDX].copy()
    g = data[:,:,G_IDX].copy()
    b = data[:,:,B_IDX].copy()

    r = (r * gains[R_IDX]).astype(np.uint8)
    g = (g * gains[G_IDX]).astype(np.uint8)
    b = (b * gains[B_IDX]).astype(np.uint8)
    
    r += (gamma * (r - g)).astype(np.uint8)
    b += (gamma * (b - g)).astype(np.uint8)

    r[r < 0] = 0
    r[r > 255] = 255
    g[g < 0] = 0
    g[g > 255] = 255
    b[b < 0] = 0
    b[b > 255] = 255
    
    result = np.empty(data.shape, dtype=np.uint8)
    result[:,:,0] = r
    result[:,:,1] = g
    result[:,:,2] = b
    
    return result
    
def scale_rgb(data, sigma=10, min=0, max=1, gains=[0.9,1.1,1.8], gamma=0.1):
    R_IDX = 0
    G_IDX = 1
    B_IDX = 2
    
    if min < 0:
        data = data - min
        max = max - min
        min = 0

    r = data[R_IDX].copy()
    g = data[G_IDX].copy()
    b = data[B_IDX].copy()

    slope = 255 / arcsinh((max - min)/sigma)

    mean = (r + g + b)/3
    mean[mean < min] = 0
    r[mean == 0] = 0
    g[mean == 0] = 0
    b[mean == 0] = 0
    
    scale = slope * arcsinh((mean - min) / sigma) / mean

    r = (r * scale).astype(int)
    g = (g * scale).astype(int)
    b = (b * scale).astype(int)
    
    r = (r * gains[R_IDX]).astype(int)
    g = (g * gains[G_IDX]).astype(int)
    b = (b * gains[B_IDX]).astype(int)
    
    r += (gamma * (r - g)).astype(int)
    b += (gamma * (b - g)).astype(int)

    r[r < 0] = 0
    r[r > 255] = 255
    g[g < 0] = 0
    g[g > 255] = 255
    b[b < 0] = 0
    b[b > 255] = 255
    
    result = np.empty(data.shape, dtype=np.uint8)
    result[0] = r
    result[1] = g
    result[2] = b

    return result

In [ ]:
a_data.min()

In [ ]:
a_data.max()

In [ ]:
a_data.shape

In [ ]:
nan_data = a_data.copy()
nan_data[np.logical_not(np.isnan(nan_data))] = 0
nan_data[np.isnan(nan_data)] = 1
#print(nan_data.shape)

output = np.empty((2050,1503,3))
output[:,:,0] = nan_data[0]
output[:,:,1] = nan_data[1]
output[:,:,2] = nan_data[2]

In [ ]:
#r, g, b = scale_rgb(a_data/a_data.max(), min=0, max=1)

In [ ]:
r = a_data[0]
g = a_data[1]
b = a_data[2]

In [ ]:
r.shape

In [ ]:
rgb_default = make_lupton_rgb(r, g, b, filename='rgb-000756-2-0427.png', Q=8, stretch=0.4)
plt.figure(figsize=(20,40))
plt.grid()
plt.imshow(rgb_default, origin='lower')


In [ ]:
def cutout(data, wcs, ra, dec, x_size=100, y_size=100):
    x_centre, y_centre = world2pix(ra, dec, wcs)
    x_top = int(round(x_centre) - x_size/2)
    y_top = int(round(y_centre) - y_size/2)
    x_bottom = x_top + x_size
    y_bottom = y_top + y_size
    
    return data[y_top:y_bottom, x_top:x_bottom]

def world2pix(ra, dec, wcs):
    coords = np.array([[ra, dec]])
    location = wcs.wcs_world2pix(coords, 0, ra_dec_order=True)
    return location[0][0], location[0][1]

In [ ]:
r_cutout = cutout(r, WCS(a_header), 179.689293428, -0.454379058, x_size=224, y_size=224)
g_cutout = cutout(g, WCS(a_header), 179.689293428, -0.454379058, x_size=224, y_size=224)
b_cutout = cutout(b, WCS(a_header), 179.689293428, -0.454379058, x_size=224, y_size=224)

r_cutout[r_cutout < 0] = 0
g_cutout[g_cutout < 0] = 0
b_cutout[b_cutout < 0] = 0

vmax = max(r.max(), g.max(), b.max())

In [ ]:
rgb_cutout = make_lupton_rgb(r_cutout, g_cutout, b_cutout, filename='rgb-cutout-000756-2-0427.png', Q=8, stretch=0.5)
#plt.figure(figsize=(5,5))
plt.grid(False)
plt.imshow(rgb_cutout, origin='lower')

In [ ]:
from astropy.convolution import Gaussian2DKernel
from astropy.convolution import convolve

kernel = Gaussian2DKernel(x_stddev=0.3)
rgb_convoled = rgb_cutout.copy()

rgb_convoled[:,:,0] = convolve(rgb_cutout[:,:,0], kernel)
rgb_convoled[:,:,1] = convolve(rgb_cutout[:,:,1], kernel)
rgb_convoled[:,:,2] = convolve(rgb_cutout[:,:,2], kernel)

#plt.figure(figsize=(5,5))
plt.grid(False)
plt.imshow(rgb_convoled, origin='lower')

In [ ]:
(rgb_cutout[:,:,1]).shape

In [ ]:
from astropy.coordinates import SkyCoord
import astropy.units as u
from astropy.nddata import Cutout2D

In [ ]:
position = SkyCoord(ra=179.689293428*u.deg, dec=-0.454379058*u.deg, frame='icrs')
cutout_size = u.Quantity((25,25), u.arcsec)
r_cutout_1 = Cutout2D(r,position=position,size=cutout_size, wcs=WCS(twod_fits[0].header))
g_cutout_1 = Cutout2D(g,position=position,size=cutout_size, wcs=WCS(twod_fits[0].header))
b_cutout_1 = Cutout2D(b,position=position,size=cutout_size, wcs=WCS(twod_fits[0].header))

In [ ]:
# r_cutout_data = convolve(r_cutout_1.data, kernel)
# g_cutout_data = convolve(g_cutout_1.data, kernel)
# b_cutout_data = convolve(b_cutout_1.data, kernel)

r_cutout_data = r_cutout_1.data
g_cutout_data = g_cutout_1.data
b_cutout_data = b_cutout_1.data

r_cutout_data[r_cutout_data < 0] = 0
g_cutout_data[g_cutout_data < 0] = 0
b_cutout_data[b_cutout_data < 0] = 0

rgb = np.empty((3, r_cutout_data.shape[0], r_cutout_data.shape[1]))
print(rgb.shape)
rgb[0] = r_cutout_data
rgb[1] = g_cutout_data
rgb[2] = b_cutout_data

#vmax = rgb.max()

#modified_cutout = scale_rgb(rgb,sigma=1/10,max=vmax)

modified_cutout = make_lupton_rgb(rgb[0]/vmax,rgb[1]/vmax,rgb[2]/vmax, filename='rgb-cutout-000756-2-0427.png', Q=0.00001, stretch=0.2)

# rgb_cutout = make_lupton_rgb(r_cutout_data/vmax, g_cutout_data/vmax, b_cutout_data/vmax, filename='rgb-cutout-000756-2-0427.png', Q=10, stretch=0.2)

# modified_cutout[:,:,0] = convolve(modified_cutout[:,:,0], kernel)
# modified_cutout[:,:,1] = convolve(modified_cutout[:,:,1], kernel)
# modified_cutout[:,:,2] = convolve(modified_cutout[:,:,2], kernel)

plt.figure(figsize=(5,5))
plt.grid(False)
plt.imshow(modified_cutout, origin='lower')

In [ ]:
from PIL import Image
from PIL import ImageFilter

#modified_cutout.dtype

rescaled_image = Image.fromarray(modified_cutout).resize((512,512), Image.BICUBIC).transpose(Image.FLIP_TOP_BOTTOM)
rescaled_image = rescaled_image.filter(ImageFilter.GaussianBlur())
plt.figure(figsize=(10,10))
plt.grid(False)
plt.imshow(rescaled_image)

In [ ]:
rescaled_image.save('rescaled_cutout.jpeg')

In [ ]:
vmax

In [ ]:
wcs = WCS(a_header)
ra, dec, petroRad = (179.686410, -0.602918, 3.719159)

In [2]:
import numpy as np
petroRad = 3.719159
nRp = 5 # 2 x 3Rp. 2Rp can still miss some of elliptical galaxies
minScaling = 0.9
rotationFactor = np.sqrt(2)

angularsize = nRp * rotationFactor / minScaling * petroRad
angularsize

29.220472769010875

In [ ]:
from astropy.nddata import Cutout2D
from astropy.coordinates import SkyCoord, ICRS
import astropy.units as u
from astropy.units import Quantity

In [ ]:
coords = SkyCoord(ra=ra, dec=dec, frame=ICRS, unit=u.deg)

In [ ]:
size = Quantity(angularsize, unit=u.arcsec)

In [ ]:
cutout_r = Cutout2D(r, position=coords, size=size, wcs=wcs).data
cutout_g = Cutout2D(g, position=coords, size=size, wcs=wcs).data
cutout_b = Cutout2D(b, position=coords, size=size, wcs=wcs).data

In [ ]:
cutout = np.empty((3, cutout_r.shape[0], cutout_r.shape[1]))
cutout[0] = cutout_r
cutout[1] = cutout_g
cutout[2] = cutout_b

In [ ]:
scaled_data = scale_rgb(cutout, max=np.max(cutout), sigma=1/2.5)

plt.imshow(np.moveaxis(scaled_data, 0, -1), interpolation='bicubic', origin='lower')

In [ ]:
def resize(data, size):
    output = np.empty((3, size, size))
    output[0] = np.array(Image.fromarray(data[0]).resize((size, size), Image.BICUBIC))
    output[1] = np.array(Image.fromarray(data[1]).resize((size, size), Image.BICUBIC))
    output[2] = np.array(Image.fromarray(data[2]).resize((size, size), Image.BICUBIC))
    return output

In [ ]:
img64 = resize(cutout, 64)
img64.shape

In [ ]:
scaled_img64 = scale_rgb(img64, max=np.max(img64), sigma=1/2.5)

plt.imshow(np.moveaxis(scaled_img64, 0, -1), interpolation='bicubic', origin='lower')

In [ ]:
resultant_cutoutsize = int(cutout_b.shape[0] / rotationFactor)
resultant_cutoutsize

In [ ]:
cutout.shape

In [ ]:
cutout_size = cutout_b.shape[0]
start = int((cutout_size - resultant_cutoutsize) / 2)
end = start + resultant_cutoutsize

imgRes = cutout[:,start:end,start:end]
#imgRes = resize(cutout, resultant_cutoutsize)
# scale_rgb(cutout, min=np.min(cutout), max=np.max(cutout), sigma=1/2.5)
scaled_imgRes = scale_rgb(imgRes, max=np.max(imgRes), sigma=1/2.5)
print(np.max(scaled_imgRes))
resized_imgRes = resize(scaled_imgRes/255.0, 224)

plt.imshow(np.moveaxis(resized_imgRes, 0, -1), interpolation='bicubic', origin='lower')